In [14]:
import json
import os
from datetime import datetime
from pathlib import Path
from typing import Final, Literal

import numpy as np
import structlog
import torch
import torch.nn as nn
from numpy import argmax, concatenate, ndarray, vstack, where
from pandas import DataFrame, Series, concat, read_csv, set_option, to_datetime
from scipy.spatial.distance import pdist, squareform
from scipy.stats import loguniform, uniform
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.metrics import (
	auc,
	average_precision_score,
	classification_report,
	f1_score,
	precision_recall_curve,
	roc_auc_score,
)
from sklearn.model_selection import ParameterSampler
from sklearn.preprocessing import RobustScaler
from sklearn.svm import OneClassSVM

set_option("display.max_columns", None)
logger: Final[structlog.stdlib.BoundLogger] = structlog.get_logger(__name__)
type ParamGrid = dict[str, tuple[float | str, ...]]

COLUMNS: Final[list[str]] = [
	"timestamp",
	"activity",
	"heart_rate",
	*[
		f"IMU_{body_part}_{suffix}"
		for body_part in ["hand", "chest", "ankle"]
		for suffix in [
			"temp_C",
			*[
				f"{scalar}_{axis}"
				for scalar in ["acc16g_ms^-2", "acc6g_ms^-2", "gyro_rad/s", "mag_μT"]
				for axis in ["x", "y", "z"]
			],
			*[f"orient_{x}" for x in range(1, 5)],
		]
	],
]
IMU_COLUMNS: Final[list[str]] = [
	col
	for col in COLUMNS
	if col.startswith("IMU_") and "acc6g_ms^-2" not in col and "orient" not in col
]

In [15]:
class LSTMAutoencoder(nn.Module):
	"""LSTM-based Autoencoder for time-series novelty detection."""

	def __init__(
		self,
		n_features: int,
		sequence_length: int,
		hidden_size: int = 64,
		n_layers: int = 2,
		dropout: float = 0.2,
	):
		super(LSTMAutoencoder, self).__init__()
		self.n_features = n_features
		self.sequence_length = sequence_length
		self.hidden_size = hidden_size
		self.encoder = nn.LSTM(
			input_size=n_features,
			hidden_size=hidden_size,
			num_layers=n_layers,
			batch_first=True,
			dropout=dropout if n_layers > 1 else 0,
		)
		self.decoder = nn.LSTM(
			input_size=hidden_size,
			hidden_size=hidden_size,
			num_layers=n_layers,
			batch_first=True,
			dropout=dropout if n_layers > 1 else 0,
		)
		self.output_layer = nn.Linear(hidden_size, n_features)

	def forward(self, x):
		# Encode
		_, (hidden, cell) = self.encoder(x)
		# Repeat hidden state for decoder
		decoder_input = hidden[-1].unsqueeze(1).repeat(1, self.sequence_length, 1)
		# Decode
		decoder_output, _ = self.decoder(decoder_input, (hidden, cell))
		# Output reconstruction
		return self.output_layer(decoder_output)


class LSTMAutoencoderWrapper:
	"""Wrapper for LSTM Autoencoder to match sklearn API."""

	def __init__(
		self,
		n_features: int,
		sequence_length: int = 100,
		hidden_size: int = 64,
		n_layers: int = 2,
		dropout: float = 0.2,
		epochs: int = 50,
		batch_size: int = 64,
		learning_rate: float = 1e-3,
		device: str | None = None,
	):
		self.n_features = n_features
		self.sequence_length = sequence_length
		self.hidden_size = hidden_size
		self.n_layers = n_layers
		self.dropout = dropout
		self.epochs = epochs
		self.batch_size = batch_size
		self.learning_rate = learning_rate
		self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

		self.model = None
		self.threshold = None

	def _create_sequences(self, data: ndarray) -> ndarray:
		"""Create sliding windows from flat data."""
		data = data.astype(np.float32)
		return np.array(
			[
				data[i : i + self.sequence_length]
				for i in range(len(data) - self.sequence_length + 1)
			],
			dtype=np.float32,  # Explicitly set dtype
		)

	def fit(self, X: DataFrame, y=None):
		"""Train the LSTM Autoencoder on normal data."""
		# Convert to numpy
		sequences = self._create_sequences(X[IMU_COLUMNS].values.astype(np.float32))
		if len(sequences) == 0:
			logger.warning(
				"Not enough data to create sequences. Skipping LSTM-AE training."
			)
			return self

		self.model = LSTMAutoencoder(
			n_features=self.n_features,
			sequence_length=self.sequence_length,
			hidden_size=self.hidden_size,
			n_layers=self.n_layers,
			dropout=self.dropout,
		).to(self.device)

		# Training setup
		criterion = nn.MSELoss()
		optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
		dataloader = torch.utils.data.DataLoader(
			torch.utils.data.TensorDataset(torch.FloatTensor(sequences)),
			batch_size=self.batch_size,
			shuffle=True,
		)
		self.model.train()
		for epoch in range(self.epochs):  # Training loop
			epoch_loss = 0
			for batch in dataloader:
				batch_data = batch[0].to(self.device)
				# Forward pass
				reconstruction = self.model(batch_data)
				loss = criterion(reconstruction, batch_data)
				# Backward pass
				optimizer.zero_grad()
				loss.backward()
				optimizer.step()

				epoch_loss += loss.item()

			if (epoch + 1) % 10 == 0:
				logger.info(
					f"LSTM-AE Epoch {epoch + 1}/{self.epochs} "
					+ f"Loss: {epoch_loss / len(dataloader):.4f}",
				)

		# Treshold on training data (95th percentile of reconstruction errors)
		self.threshold = np.percentile(self._calc_reconstruction_errors(sequences), 95)

		return self

	def _calc_reconstruction_errors(self, sequences: ndarray) -> ndarray:
		"""Compute reconstruction error for sequences."""
		assert self.model is not None, "Model is not trained yet."
		self.model.eval()
		errors = []

		with torch.no_grad():
			for i in range(0, len(sequences), self.batch_size):
				batch_tensor = torch.FloatTensor(sequences[i : i + self.batch_size]).to(
					self.device
				)
				errors.extend(  # MSE per sequence
					torch.mean(
						(batch_tensor - self.model(batch_tensor)) ** 2, dim=(1, 2)
					)
					.cpu()
					.numpy()
				)
		return np.array(errors)

	def predict(self, X: DataFrame) -> ndarray:
		"""Predict: 1 for normal, -1 for anomaly."""
		return where(self.decision_function(X) > self.threshold, -1, 1)

	def decision_function(self, X: DataFrame) -> ndarray:
		"""Return anomaly scores (higher = more anomalous)."""
		sequences = self._create_sequences(X[IMU_COLUMNS].values.astype(np.float32))
		# Data is already scaled - no transformation needed
		if len(sequences) == 0:
			# If not enough data for a sequence, return high anomaly score
			return np.array([self.threshold * 2 if self.threshold else 1.0] * len(X))

		errors = self._calc_reconstruction_errors(sequences)
		# Pad to match original length (for sequences lost at the end)
		if (padding := len(X) - len(errors)) > 0:
			errors = np.concatenate([errors, errors[-1:].repeat(padding)])

		return errors

	def get_params(self, deep=True):
		"""
		Get parameters for sklearn compatibility.

		Args:
			deep (bool): Ignored, present for sklearn compatibility. Default to True.
		"""
		return {
			"n_features": self.n_features,
			"sequence_length": self.sequence_length,
			"hidden_size": self.hidden_size,
			"n_layers": self.n_layers,
			"dropout": self.dropout,
			"epochs": self.epochs,
			"batch_size": self.batch_size,
			"learning_rate": self.learning_rate,
		}

	def set_params(self, **params):
		"""Set parameters for sklearn compatibility."""
		for key, value in params.items():
			setattr(self, key, value)
		return self

In [16]:
class NoveltyDetectionEnsemble:
	"""Ensemble combining LSTM-AE, Isolation Forest, and One-Class SVM."""

	def __init__(
		self,
		n_features: int,
		# LSTM-AE params
		sequence_length: int = 100,
		hidden_size: int = 64,
		n_layers: int = 2,
		dropout: float = 0.2,
		epochs: int = 50,
		batch_size: int = 64,
		learning_rate: float = 1e-3,
		# Isolation Forest params
		n_estimators: int = 200,
		max_samples: int = 256,
		contamination: float = 0.05,
		# OC-SVM params
		nu: float = 0.05,
		gamma: float | Literal["scale", "auto"] = "scale",
		# Ensemble params
		# weights: tuple[float, float, float] = (0.4, 0.3, 0.3),
		weights: tuple[float, float] = (0.3, 0.3),
	):
		self.n_features = n_features
		self.weights = np.array(weights)

		self.lstm_ae = LSTMAutoencoderWrapper(
			n_features=n_features,
			sequence_length=sequence_length,
			hidden_size=hidden_size,
			n_layers=n_layers,
			dropout=dropout,
			epochs=epochs,
			batch_size=batch_size,
			learning_rate=learning_rate,
		)
		self.isolation_forest = IsolationForest(
			n_estimators=n_estimators,
			max_samples=max_samples,
			contamination=contamination,
			random_state=42,
		)
		self.ocsvm = OneClassSVM(nu=nu, gamma=gamma, kernel="rbf")

		self.models = [self.isolation_forest, self.ocsvm]
		self.model_names = ["IsolationForest", "OC-SVM"]

	def fit(self, X: DataFrame, y=None):
		"""Train all models in the ensemble."""
		logger.info("Training ensemble models...")
		for name, model in zip(self.model_names, self.models, strict=True):
			logger.info(f"Training {name}...")
			model.fit(X, y)
		logger.info("Ensemble training complete.")
		return self

	def predict(self, X: DataFrame) -> ndarray:
		"""Predict: 1 for normal, -1 for anomaly."""
		scores = self.decision_function(X)
		return where(scores > np.median(scores), -1, 1)

	def decision_function(self, X: DataFrame) -> ndarray:
		"""Return combined anomaly scores (higher = more anomalous)."""
		return np.average(
			[
				# self._normalize_scores(self.lstm_ae.decision_function(X)),
				self._normalize_scores(
					-self.isolation_forest.decision_function(X.values)
				),
				self._normalize_scores(-self.ocsvm.decision_function(X.values)),
			],
			axis=0,
			weights=self.weights,
		)

	def _normalize_scores(self, scores: ndarray) -> ndarray:
		"""Normalize scores to [0, 1] range using min-max scaling."""
		if (max_score := np.max(scores)) - (min_score := np.min(scores)) == 0:
			return np.zeros_like(scores)
		return (scores - min_score) / (max_score - min_score)

	def get_params(self, deep=True):
		"""Get all ensemble parameters."""
		params = {"n_features": self.n_features, "weights": tuple(self.weights)}
		for model in self.models:
			params.update({f"{k}": v for k, v in model.get_params(deep=deep).items()})
		return params

In [17]:
def log_iteration_results(
	log_file: Path,
	iteration: int,
	metrics: dict,
	hpo_method: str,
) -> None:
	"""
	Log results from a single HPO iteration using structured logging format.

	Args:
		log_file: Path to log file
		iteration: Current iteration number
		metrics: Dictionary containing metrics, params, and datetime
		hpo_method: Name of HPO method being used
	"""
	with open(log_file, "a") as f:
		f.write(
			json.dumps(
				{
					"event": {
						"iteration": iteration,
						"fold": metrics.get("fold", 0),
						"target": metrics.get("f1_score", 0.0),
						"avg_precision": metrics.get("avg_precision", 0.0),
						"auc_pr": metrics.get("auc_pr", 0.0),
						"auc_roc": metrics.get("auc_roc", 0.0),
						"params": metrics.get("params", {}),
						"datetime": metrics.get("datetime", ""),
					},
					"logger": hpo_method,
					"level": "info",
					"timestamp": datetime.now().isoformat() + "Z",
				}
			)
			+ "\n"
		)


def score_function(
	model: NoveltyDetectionEnsemble,
	train: DataFrame,
	test: Series,
	testing_data: DataFrame,
) -> dict[str, float | int | str | dict]:
	"""
	Objective function to maximize, calcs the F1 score on the test set.
	Now works with ensemble model.

	Args:
		model (NoveltyDetectionEnsemble): Ensemble model to eval
		Train (DataFrame): train data, only for API compliance
		test (Series): true targets (True for novel, False for normal)
		testing_data (DataFrame): test feature data

	Returns:
		dict: Evaluation metrics
	"""
	# Get predictions and anomaly scores
	f1 = f1_score(test, where(model.predict(testing_data) == -1, True, False))
	# Get decision scores (higher values = more anomalous)
	anomaly_scores = model.decision_function(testing_data)
	precision, recall, _ = precision_recall_curve(test, anomaly_scores)
	metrics = {
		"f1_score": float(f1),
		"avg_precision": float(average_precision_score(test, anomaly_scores)),
		"auc_pr": float(auc(recall, precision)),
		"auc_roc": float(roc_auc_score(test, anomaly_scores)),
		"params": model.get_params(),
		"datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
	}
	logger.info(f"Evaluation metrics: {metrics}")
	return metrics

In [18]:
class IncrementalDataset:
	"""Manages data splits for incremental/continual learning experiments."""

	def __init__(
		self,
		path: Path,
		initial_activities: int = 3,
		test_size: int = 2,
		activity_order: list[int] | None = None,
		ordering_method: Literal["pca", "statistical", "variance", "frequency"] = "pca",
	):
		self.data, self.labels = self.load_data(path)
		self.df = self.data.merge(self.labels, how="left", on="id")

		self.initial_activities = initial_activities
		self.test_size = test_size

		self.activity_order = (
			self.get_activity_order(self.data, self.labels, method=ordering_method)
			if activity_order is None
			else activity_order
		)
		self.n_activities = len(self.activity_order)
		self.n_folds = (self.n_activities - initial_activities) // test_size

	def read_w_log(self, path: Path, filename: str) -> tuple[DataFrame, str]:
		"""Read and preprocess PAMAP2 data file."""
		print(f"Reading: {filename}", end="\r")
		df = read_csv(os.path.join(path, filename), sep=r"\s+", header=None)
		df.columns = COLUMNS
		return (
			df.loc[:, ~df.columns.str.contains(r"orient|acc6g", regex=True)],
			filename.split(".")[0][-2:],
		)

	def handle_nans(self, df: DataFrame) -> DataFrame:
		"""Handles NaN values in the sensor data with a time-series-aware strategy."""
		df = df.copy()
		for col in IMU_COLUMNS:
			df.loc[:, col] = (
				df[col]
				.ffill(limit=2)
				.interpolate("linear", limit=5, limit_direction="both")
			)
		return df.dropna(subset=IMU_COLUMNS)

	def normalize_features(
		self,
		X_train: DataFrame,
		X_val: DataFrame | None = None,
		X_test: DataFrame | None = None,
		scaler: RobustScaler | None = None,
		force_refit: bool = False,
	) -> tuple[DataFrame, DataFrame | None, DataFrame | None, RobustScaler]:
		"""Normalizes IMU features using RobustScaler."""
		if scaler is None or force_refit:
			scaler = RobustScaler().fit(X_train[IMU_COLUMNS])

		X_train = X_train.copy()
		X_train.loc[:, IMU_COLUMNS] = scaler.transform(X_train[IMU_COLUMNS])

		X_val_norm = None
		if X_val is not None:
			X_val_norm = X_val.copy()
			X_val_norm.loc[:, IMU_COLUMNS] = scaler.transform(X_val[IMU_COLUMNS])

		X_test_norm = None
		if X_test is not None:
			X_test_norm = X_test.copy()
			X_test_norm.loc[:, IMU_COLUMNS] = scaler.transform(X_test[IMU_COLUMNS])

		return X_train, X_val_norm, X_test_norm, scaler

	def load_data(self, base_path: Path) -> tuple[DataFrame, DataFrame]:
		"""Load and preprocess all PAMAP2 protocol data."""
		logger.info("Loading PAMAP2 data...")
		DATA_DIR = Path("../data/PAMAP2/data.csv")
		LABELS_DIR = Path("../data/PAMAP2/labels.csv")
		if DATA_DIR.exists() and LABELS_DIR.exists():
			logger.info("Found preprocessed data files. Loading...")
			return read_csv(DATA_DIR), read_csv(LABELS_DIR)

		data, labels = [], []
		for df, subject in [
			self.read_w_log(base_path, filename)
			for filename in os.listdir(base_path)
			if filename.endswith(".dat")
		]:
			df = self.handle_nans(df[~df["activity"].isin([0, 24])])
			df["subject"] = str(subject)
			df["timestamp"] = to_datetime(df["timestamp"], unit="s").dt.time
			df["id"] = df["subject"] + "_" + df["timestamp"].astype(str)

			data.append(
				df.drop(columns=["timestamp", "heart_rate", "activity", "subject"])
			)
			labels.append(df[["id", "timestamp", "activity", "subject"]])

		data, labels = concat(data), concat(labels)
		data["subject"] = data["subject"].astype("category")
		labels["activity"] = labels["activity"].astype("category")

		data.to_csv(DATA_DIR, index=False)
		labels.to_csv(LABELS_DIR, index=False)

		return data, labels

	def get_activity_order(
		self,
		data: DataFrame,
		labels: DataFrame,
		method: Literal["pca", "statistical", "variance", "frequency"] = "pca",
	) -> list[int]:
		"""Order activities by distinctiveness for incremental learning."""
		df = data.merge(labels, how="left", on="id")

		if method == "frequency":
			return df["activity"].value_counts().index.tolist()

		activity_stats, activity_counts = {}, {}
		for activity in df["activity"].unique():
			activity_data = df[df["activity"] == activity][IMU_COLUMNS]
			activity_counts[activity] = len(activity_data)

			if method == "pca":
				activity_stats[activity] = (
					PCA(n_components=min(10, len(IMU_COLUMNS), len(activity_data)))
					.fit_transform(activity_data)
					.mean(axis=0)
				)
			elif method == "statistical":
				activity_stats[activity] = concatenate(
					[
						activity_data.mean().values,
						activity_data.std().values,
						activity_data.quantile(0.25).values,
						activity_data.quantile(0.75).values,
					]
				)
			elif method == "variance":
				activity_stats[activity] = concatenate(
					[
						activity_data.var().values,
						activity_data.abs().mean().values,
					]
				)
		activities = list(activity_stats.keys())
		distances = squareform(
			pdist(
				vstack([activity_stats[act] for act in activities]), metric="euclidean"
			)
		)
		ordered = []
		remaining = set(range(len(activities)))
		avg_distances = distances.mean(axis=1)
		first_idx = argmax(
			[
				-avg_distances[i] if i in remaining else float("-inf")
				for i in range(len(activities))
			]
		).astype(int)
		ordered.append(first_idx)
		remaining.remove(first_idx)

		while remaining:
			min_dists = [
				min([distances[idx, sel_idx] for sel_idx in ordered])
				if idx in remaining
				else float("inf")
				for idx in range(len(activities))
			]
			next_idx = argmax(
				[
					-min_dists[i] if i in remaining else float("-inf")
					for i in range(len(activities))
				]
			).astype(int)
			ordered.append(next_idx)
			remaining.remove(next_idx)

		return [activities[i] for i in ordered]

	def get_fold(
		self,
		fold_idx: int,
		val_split: float = 0.2,
		normalize: bool = True,
		scaler: RobustScaler | None = None,
		pollution_rate: float = 0.3,  # 30% normal activities in val/test
	) -> tuple[
		DataFrame,
		DataFrame | None,
		DataFrame | None,
		DataFrame,
		DataFrame | None,
		DataFrame,
		RobustScaler | None,
	]:
		"""
		Get train/val/test split for a specific fold with polluted val/test sets.

		Pollution means including some normal (train) activities in val/test sets
		to make evaluation more realistic.
		"""
		if fold_idx >= self.n_folds:
			raise ValueError(f"fold_idx {fold_idx} >= n_folds {self.n_folds}")

		n_train_activities = self.initial_activities + fold_idx * self.test_size
		train_activities = self.activity_order[:n_train_activities]
		test_activities = self.activity_order[
			n_train_activities : n_train_activities + self.test_size
		]
		# Get train/val data (normal activities)
		train_val_df = self.df[self.df["activity"].isin(train_activities)].copy()
		# Get test data (novel activities)
		novel_df = self.df[self.df["activity"].isin(test_activities)].copy()
		# Split train activities into train/val
		train_data, val_normal_data = [], []
		for activity in train_val_df["activity"].unique():
			activity_data = train_val_df[train_val_df["activity"] == activity]
			split_idx = int(len(activity_data) * (1 - val_split))
			train_data.append(activity_data.iloc[:split_idx])
			val_normal_data.append(activity_data.iloc[split_idx:])

		train_df = concat(train_data).reset_index(drop=True)
		# Create polluted validation set: mix of normal and novel
		val_normal = concat(val_normal_data).reset_index(drop=True)

		# Sample novel activities for validation
		val_novel = novel_df.sample(
			n=min(
				int(len(val_normal) * (1 - pollution_rate) / pollution_rate),
				len(novel_df) // 2,
			),
			random_state=42,
		).reset_index(drop=True)

		val_df = (
			concat([val_normal, val_novel])
			.sample(frac=1, random_state=42)
			.reset_index(drop=True)
		)
		# polluted test set: mix of normal and novel usinfg remaining novel
		# samples not used in validation
		test_novel = (
			novel_df[~novel_df.index.isin(val_novel.index)].reset_index(drop=True)
			if val_df is not None
			else novel_df
		)
		# Sample normal activities for test (from train set, not val)
		test_normal = train_df.sample(
			n=min(
				int(len(test_novel) * pollution_rate / (1 - pollution_rate)),
				len(train_df) // 10,
			),
			random_state=42,
		).reset_index(drop=True)
		test_df = (
			concat([test_normal, test_novel])
			.sample(frac=1, random_state=42)
			.reset_index(drop=True)
		)
		feature_cols = [col for col in self.data.columns if col != "id"]

		X_train = train_df[feature_cols].reset_index(drop=True)
		X_val = (
			val_df[feature_cols].reset_index(drop=True) if val_df is not None else None
		)
		X_test = test_df[feature_cols].reset_index(drop=True)

		y_train = train_df[["id", "activity"]].reset_index(drop=True)
		y_train["isNovelty"] = False  # all normal

		if val_df is not None:
			y_val = val_df[["id", "activity"]].reset_index(drop=True)
			# Mark as novel if not in train activities
			y_val["isNovelty"] = ~y_val["activity"].isin(train_activities)
		else:
			y_val = None

		y_test = test_df[["id", "activity"]].reset_index(drop=True)
		# Mark as novel if not in train activities
		y_test["isNovelty"] = ~y_test["activity"].isin(train_activities)

		if normalize:
			X_train, X_val, X_test, scaler = self.normalize_features(
				X_train, X_val, X_test, scaler=scaler
			)

		if y_val is not None:
			logger.info(
				f"Fold {fold_idx} - Val set: {(~y_val['isNovelty']).sum()} normal, "
				f"{y_val['isNovelty'].sum()} novel ({y_val['isNovelty'].mean():.2%} novel)"
			)
		logger.info(
			f"Fold {fold_idx} - Test set: {(~y_test['isNovelty']).sum()} normal, "
			f"{y_test['isNovelty'].sum()} novel ({y_test['isNovelty'].mean():.2%} novel)"
		)
		return X_train, X_val, X_test, y_train, y_val, y_test, scaler

In [19]:
def get_search_space(
	use_log_dist: bool = False,
) -> dict[str, list[int | float | tuple[float]]]:
	"""
	Get hyperparameter search space for the ensemble model.

	Args:
		use_log_dist: If to log-uniform distributions for parameters that span orders of magnitude

	Returns:
		Dictionary with parameter distributions
	"""
	return {
		# LSTM-AE params
		"sequence_length": [50, 100, 150, 200],
		"hidden_size": [32, 64, 128, 256],
		"n_layers": [1, 2, 3],
		"dropout": uniform(0.0, 0.5),
		"epochs": [20, 30, 50, 75, 100],
		"batch_size": [32, 64, 128, 256],
		"learning_rate": loguniform(1e-4, 1e-2)
		if use_log_dist
		else uniform(1e-4, 1e-2),
		# Isolation Forest params
		"n_estimators": [100, 200, 300, 500],
		"max_samples": [128, 256, 512, 1024],
		"contamination": loguniform(0.001, 0.1)
		if use_log_dist
		else uniform(0.001, 0.1),
		# OC-SVM params
		"nu": loguniform(0.01, 0.2) if use_log_dist else uniform(0.01, 0.2),
		"gamma": loguniform(1e-4, 1.0) if use_log_dist else uniform(1e-4, 1.0),
		# Ensemble weights - using uniform for weights
		# "weights": [
		# 	(0.5, 0.3, 0.2),
		# 	(0.4, 0.3, 0.3),
		# 	(0.3, 0.4, 0.3),
		# 	(0.3, 0.3, 0.4),
		# 	(0.33, 0.33, 0.34),
		# ],
		"weights": [
			(0.3, 0.2),
			(0.3, 0.3),
			(0.4, 0.3),
			(0.3, 0.4),
			(0.33, 0.34),
		],
	}  # type: ignore


def get_search_method(
	search_name: Literal["random_search", "random_search_log"],
	n_iter: int = 10,
	random_state: int = 42,
) -> ParameterSampler:
	"""
	Get the appropriate search method with corresponding parameter space.

	Args:
		search_name: Name of the search method to use
		n_iter: Number of iterations for the search
		random_state: Random state for reproducibility

	Returns:
		Iterator over parameter dictionaries (ParameterSampler)
	"""
	param_distributions = get_search_space(use_log_dist="log" in search_name.lower())

	if search_name in ["random_search", "random_search_log"]:
		return ParameterSampler(  # This is an iterator over parameter settings
			param_distributions, n_iter=n_iter, random_state=random_state
		)
	raise ValueError(f"Unknown search method: {search_name}")

In [20]:
def train_and_evaluate_fold(
	dataset: IncrementalDataset,
	fold_idx: int,
	model_params: dict[str, float | int | tuple[float, ...]],
) -> dict:
	"""
	Train and evaluate the ensemble model on a specific fold.

	Args:
		dataset: IncrementalDataset instance
		fold_idx: Fold index to train/evaluate
		model_params: Optional model hyperparameters (used if hpo_method="none")

	Returns:
		Dictionary with evaluation metrics from validation set
	"""
	X_train, X_val, X_test, y_train, y_val, y_test, scaler = dataset.get_fold(
		fold_idx, val_split=0.2, normalize=True
	)
	assert X_train is not None, "Train set can't be None"
	assert X_val is not None and y_val is not None, "val set & labels can't be None"
	assert X_test is not None, "Test set can't be None"

	model_params["n_features"] = len(IMU_COLUMNS)  # sanity check

	logger.info(f"Training model on fold {fold_idx}...")
	model = NoveltyDetectionEnsemble(**model_params).fit(X_train)  # type: ignore

	metrics = score_function(model, X_train, y_val["isNovelty"], X_val)
	metrics["split"] = "validation"
	metrics["fold"] = fold_idx

	logger.info(f"Fold {fold_idx} - Validation F1: {metrics['f1_score']:.4f}")
	print(
		classification_report(
			y_val["isNovelty"],
			where(model.predict(X_test) == -1, True, False),
			target_names=["Normal", "Novel"],
		)
	)
	return metrics

In [21]:
def run_hpo_search(
	dataset: IncrementalDataset,
	hpo_method: Literal["random_search", "random_search_log"],
	n_iter: int = 10,
	output_dir: Path = Path("../results"),
) -> dict[int, dict[str, float | dict]]:
	"""
	Run HPO search across all folds for a given method.

	Args:
		dataset: IncrementalDataset instance
		hpo_method: HPO method to use
		n_iter: Number of iterations per fold
		output_dir: Directory to save results

	Returns:
		Dictionary with best results per fold
	"""
	if (log_file := output_dir / f"{hpo_method}.log").exists():
		log_file.unlink()

	best_results_per_fold: dict[int, dict[str, float | dict]] = {}

	for fold_idx in range(dataset.n_folds):
		param_sampler = get_search_method(
			search_name=hpo_method, n_iter=n_iter, random_state=42 + fold_idx
		)
		best_f1: float = -1.0
		best_params: dict | None = None
		best_metrics: dict | None = None
		# Run HPO iterations for this fold
		for iteration, params in enumerate(param_sampler):
			logger.info(f"Fold {fold_idx} - Iteration {iteration + 1}/{n_iter}")
			try:
				metrics = train_and_evaluate_fold(
					dataset=dataset, fold_idx=fold_idx, model_params=params
				)
				metrics["iteration"] = iteration + 1
				log_iteration_results(
					log_file=log_file,
					iteration=iteration + 1,
					metrics=metrics,
					hpo_method=hpo_method,
				)
				if metrics["f1_score"] > best_f1:
					best_f1 = metrics["f1_score"]
					best_params = params.copy()
					best_metrics = metrics.copy()

					logger.info(f"New best for fold {fold_idx}! F1: {best_f1:.4f}")

			except Exception as e:
				logger.error(f"Error in fold {fold_idx}: {str(e)}")
				raise e

		if best_metrics is not None and best_params is not None:
			best_results_per_fold[fold_idx] = {
				"best_f1": best_f1,
				"best_params": best_params,
				"best_metrics": best_metrics,
			}
			logger.info(f"\nFold {fold_idx} complete - Best F1: {best_f1:.4f}")

	with open(output_dir / f"{hpo_method}_summary.json", "w") as f:
		json.dump(best_results_per_fold, f, indent=2, default=str)

	return best_results_per_fold

In [ ]:
dataset = IncrementalDataset(
	Path("../data/PAMAP2_Dataset/Protocol/"),
	initial_activities=3,
	test_size=2,
	ordering_method="pca",
)
results_random = run_hpo_search(
	dataset=dataset,
	hpo_method="random_search_log",
	n_iter=10,
	output_dir=Path("../reports"),
)
results_random_log = run_hpo_search(
	dataset=dataset,
	hpo_method="random_search",
	n_iter=10,
	output_dir=Path("../reports"),
)

2025-11-16 12:07:24 [info     ] Loading PAMAP2 data...        
2025-11-16 12:07:24 [info     ] Found preprocessed data files. Loading...
2025-11-16 12:07:31 [info     ] Fold 0 - Iteration 1/10       
2025-11-16 12:07:32 [info     ] Fold 0 - Val set: 103792 normal, 144022 novel (58.12% novel)
2025-11-16 12:07:32 [info     ] Fold 0 - Test set: 41516 normal, 266340 novel (86.51% novel)
2025-11-16 12:07:33 [info     ] Training model on fold 0...   
2025-11-16 12:07:33 [info     ] Training ensemble models...   
2025-11-16 12:07:33 [info     ] Training IsolationForest...   
2025-11-16 12:07:39 [info     ] Training OC-SVM...            
